In [1]:
# Import the necessary libraries
import pandas as pd
from pathlib import Path
from textblob import TextBlob
import re

%store -r

In [2]:
# Import the restored data
# %run PrepareData.ipynb

Stored 'tsla_tweets_df' (DataFrame)
Stored 'tsla_stock_values_df' (DataFrame)
Stored 'ta_df' (DataFrame)
Stored 'tsla_sentiments_df' (DataFrame)


In [3]:
# Reset the index of the dataframe
df = tsla_tweets_df.copy()
df = df.reset_index()

In [4]:
# Clean the text
def cleanTxt(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)#Remove @mentions
    text = re.sub(r'#', '', text)# Remove '#' hash tag
    text = re.sub(r'RT[\s]+', '', text)# Remove RT
    text = re.sub(r'http?:\/\/\S+', '', text)# Remove hyperlink
    text = re.sub(r'\$[A-Za-z0-9]+', '', text)# Remove $
    text = re.sub(r'https?:\/\/\S+', '', text)# Remove hyperlink
    
    return text

df['body'] = df['body'].apply(cleanTxt)

In [5]:
# Creaet a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [6]:
# Apply the models to the cleaned data
df['Subjectivity'] = df['body'].apply(getSubjectivity)
df['Polarity'] = df['body'].apply(getPolarity)

In [8]:
# Get the daily sentimental scores
tsla_sentiments_df_textblob = df[["Subjectivity","Polarity"]].copy()
tsla_sentiments_df_textblob['Date'] = pd.to_datetime(df['post_date'].dt.date, utc=False)
tsla_sentiments_df_textblob = tsla_sentiments_df_textblob.groupby('Date').mean()

,Subjectivity,Polarity
Date,,
2015-01-01,0.400000,-0.150000
2015-01-02,0.406539,0.180581
2015-01-03,0.210845,0.046556
2015-01-04,0.201275,0.076732
2015-01-05,0.078016,0.031528


,Subjectivity,Polarity
Date,,
2019-12-28,0.386012,0.079870
2019-12-29,0.382562,0.072515
2019-12-30,0.392603,0.080258
2019-12-31,0.404880,0.081927
2020-01-01,0.415301,0.123758


In [9]:
%store tsla_sentiments_df_textblob

Stored 'tsla_sentiments_df_textblob' (DataFrame)
